In [3]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import pandas as pd
import sqlalchemy as sa
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

import database as db
from models import County, DataTrustedIdentifier

## Database connection

In [5]:
engine, session = db.connect()
engine.echo = False

2023-03-24 16:26:36,699 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-24 16:26:36,699 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 16:26:36,701 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-24 16:26:36,702 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 16:26:36,703 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-24 16:26:36,703 INFO sqlalchemy.engine.Engine [raw sql] {}


## Replace unenconded character

In [6]:
list_unenconded_characters = {
    'error': ['Ã¡', 'Ãº', 'Ã', 'Ã³', 'Ã±', 'Ã©'],
    'correct': ['á', 'ú', 'í', 'ó', 'ñ', 'é']
}

pd.DataFrame(list_unenconded_characters)

,error,correct
0,Ã¡,á
1,Ãº,ú
2,Ã,í
3,Ã³,ó
4,Ã±,ñ
5,Ã©,é


In [9]:
for column in [DataTrustedIdentifier.state_province, DataTrustedIdentifier.county]:
    list_character_error = list_unenconded_characters['error']
    list_character_correct = list_unenconded_characters['correct']
    for special_character in zip(list_character_error, list_character_correct):
        special_character_to_find=special_character[0]
        special_character_to_replace=special_character[1]
        value = sa.func.replace(column, special_character_to_find, special_character_to_replace)
        session.query(DataTrustedIdentifier)\
            .update(values={column: value}, synchronize_session=False)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "data_trusted_identifier" does not exist
LINE 1: UPDATE data_trusted_identifier SET state_province=replace(da...
               ^

[SQL: UPDATE data_trusted_identifier SET state_province=replace(data_trusted_identifier.state_province, %(replace_1)s, %(replace_2)s)]
[parameters: {'replace_1': 'Ã¡', 'replace_2': 'á'}]
(Background on this error at: https://sqlalche.me/e/20/f405)

## Update column country trusted if column country contains variations of word Brazil

In [ ]:
count_of_brazil_in_country_trusted = session.query(DataTrustedIdentifier) \
    .filter(DataTrustedIdentifier.country_trusted == 'Brasil') \
    .count()

print('count of Brasil in country trusted: %d' % count_of_brazil_in_country_trusted)

In [ ]:
list_variations_br = ['Brasil', 'BRASIL', 'Brasil/Bolivia', 'Brasilia', 'brazil', 'Brazil', 'BRazil', 'BRAZIL', '[Brésil]', 'Brésil']

if count_of_brazil_in_country_trusted == 0:
    records_with_variations_brasil = session.query(DataTrustedIdentifier) \
        .filter(DataTrustedIdentifier.country.in_(list_variations_br)) \
        .all()

    print('count of records with variations of Brazil: %d' % len(records_with_variations_brasil))

    session.query(DataTrustedIdentifier) \
        .filter(DataTrustedIdentifier.country.in_(list_variations_br)) \
        .update({'country_trusted': 'Brasil'}, synchronize_session=False)

## Find records Brazilian's state and county. After that update column country trusted

In [ ]:
query = session.query(County).distinct().all()

uf_unaccented_lower, state_unaccented_lower, county_unaccented_lower = db.get_list_uf_state_county(query)

session.query(DataTrustedIdentifier) \
    .filter(sa.and_(DataTrustedIdentifier.country_trusted.is_(None),
                 sa.or_(uf_unaccented_lower, state_unaccented_lower),
                 county_unaccented_lower)) \
    .update({'country_trusted': 'Brasil'}, synchronize_session=False)

## Close connection

In [ ]:
session.close()
engine.dispose()